In [ ]:
#Importing Libraries 
import pandas as pd

In [ ]:
#Year Data is too big so read the data in chunks in order to get a list of chunks of the data
data_iterator = pd.read_csv("2020Data_Ready.csv", chunksize=100000)

chunk_list = []  

# Each chunk is in dataframe format
for data_chunk in data_iterator:  
    chunk_list.append(data_chunk)
    

In [ ]:
#If file contains all publishers and stats are required for all publishers you can skip this step 
#If you want stats on a specific publisher and the data includes all then run this step 
#Choosing Guilford for Template 
df = df.loc[df['name'] == 'Guilford']

In [ ]:
#Third Party and Fulfillment stats 
cl_stat1 = [] 
for chunk in chunk_list: 
    d = chunk[['Provider_Name', 'name' , 'ean_13' ,'sold_by_third_party', 'fulfilled_by_third_party', 'Month' ]]
    cl_stat1.append(d)
    
stat1_df = pd.concat(cl_stat1) #Concat data    

In [ ]:
#A List of Providers 
providers = list(stat1_df['Provider_Name'].unique())

In [ ]:
Stats1_Data = [] 
for prov in providers: 
    dp = stat1_df.loc[stat1_df['Provider_Name']  == prov]
    dp_s = dp.loc[dp['sold_by_third_party'] == 't']
    
    months = list(dp_s['Month'].unique())
    
    numb_products_fTTrue = [] 
    numb_products_fTFalse = [] 
    
    for month in months: 
        dp_s_d = dp_s.loc[dp_s['Month'] == month]
        
        #Fulfilled By Third Party True 
        dt = dp_s_d.loc[dp_s_d['fulfilled_by_third_party'] == 't']
        nt = len(dt['ean_13'].unique())
        numb_products_fTTrue.append(nt)
        
        #Fulfilled by Third Party False
        dff = dp_s_d.loc[dp_s_d['fulfilled_by_third_party'] == 'f']
        nf = len(dff['ean_13'].unique())
        numb_products_fTFalse.append(nf)
        
    stats_p = pd.DataFrame({'Month': months, 
                            'Products Sold by Third Party & Fulffiled by Third Party': numb_products_fTTrue, 
                       'Products Sold by Third Party & Not Fulffiled by Third Party' : numb_products_fTFalse})
    
    stats_p['Provider'] = prov
    
    Stats1_Data.append(stats_p)
    
    print(str(prov) + '  is done')

In [ ]:
#Concat Data and adding Domain 
Stat1_final = pd.concat(Stats1_Data)
Stat1_final['Domain'] = 'All Publishers'
Stat1_final.to_csv('2020_ThirdParty.csv' , index = False)

In [ ]:
cl_stat2 = [] 
for chunk in chunk_list: 
    d = chunk[['Provider_Name', 'name' , 'Month' , 'list_price', 'for_sale_price' ]]
    cl_stat2.append(d)
    
stat2_df = pd.concat(cl_stat2)

In [ ]:
#Divide data by provider 
providers = list(stat2_df['Provider_Name'].unique())

In [ ]:
#Stat2 % of List Price
Stats2_Data = [] 
for prov in providers: 
    
    dp = stat2_df.loc[stat2_df['Provider_Name']  == prov]
    
    months = list(dp['Month'].unique())
    
    
    avgListp = [] 
    avgSalep = [] 
    avgPercentofList = [] 
    for month in months: 
        d = dp.loc[dp['Month'] == month]
        avgListp.append(d['list_price'].mean()/ 100)
        avgSalep.append(d['for_sale_price'].mean()/ 100)

        d['SalePrice'] = d['for_sale_price'] / 100 
        d['ListPrice'] = d['list_price'] / 100 

        d['ListPrice/SalePrice'] = d['SalePrice']/d['ListPrice']
        avgPercentofList.append((d['ListPrice/SalePrice'].mean()))
        
        
    
    data2 = pd.DataFrame({'Month': months, 
                     'Avg List Price': avgListp, 
                     'Avg Sale Price': avgSalep, 
                     'Avg % of List Price': avgPercentofList})
    
    
    data2['Avg % of List Price'] = data2['Avg % of List Price'] * 100
    
    data2['Provider'] = prov
    
    Stats2_Data.append(data2)
    
    print(str(prov) + '  is done')

In [ ]:
#Concat Data and adding Domain 
Stat2_final = pd.concat(Stats2_Data)
Stat2_final['Domain'] = 'All Publishers'
Stat2_final.to_csv('2020_ListPrice.csv' , index = False)

In [ ]:
#Saving High Priority data 
with pd.ExcelWriter('AllPublishers_Statistics_DS79_2020Data.xlsx') as writer:  
    Stat1_final.to_excel(writer, sheet_name='Third Party Seller')
    Stat2_final.to_excel(writer, sheet_name='Avg % of List Price')
    #Stat3_final.to_excel(writer, sheet_name='Number of New Reviews')

## Low Priority 

In [ ]:
cl_stat3 = [] 
for chunk in chunk_list: 
    d = chunk[['Provider_Name', 'name' , 'Month' , 'ean_13', 'found' ]]
    cl_stat3.append(d)
    
    
stat3_df = pd.concat(cl_stat3)

In [ ]:
#Divide data by provider 
providers = list(stat3_df['Provider_Name'].unique())

In [ ]:
#Total unique products not found by provider 
Stats3_Data = [] 
for prov in providers: 
    dp = stat3_df.loc[stat3_df['Provider_Name']  == prov]
    
    dp_f = dp.loc[dp['found'] == 'f']
    
    months = list(dp_f['Month'].unique())
    
    numb_products = [] 
    
    for month in months: 
        dp_s_d = dp_f.loc[dp_f['Month'] == month]
        
        n = len(dp_s_d['ean_13'].unique())
        numb_products.append(n)
        
        
    stats_p = pd.DataFrame({'Month': months, 
                            'Number of Products Not Found': numb_products})
    
    stats_p['Provider'] = prov
    
    Stats3_Data.append(stats_p)
    print(str(prov) + '  is done')
    

In [ ]:
#Concat Data and adding Domain 
Stat3_final = pd.concat(Stats3_Data)
Stat3_final['Domain'] = 'All Publishers'
Stat3_final.to_csv("2020_found.csv" , index = False)

In [ ]:
#Stats 4 : Columns needed {provider name , name , month , ean13 , has buy button}

cl_stat4 = [] 
for chunk in chunk_list: 
    d = chunk[['Provider_Name', 'name' , 'Month' , 'ean_13', 'has_buy_button' ]]
    cl_stat4.append(d)
    
    
stat4_df = pd.concat(cl_stat4)

In [ ]:
#Divide data by provider 
providers = list(stat4_df['Provider_Name'].unique())

In [ ]:
#Total unique products missing buy button by provider
Stats4_Data = [] 
for prov in providers: 
    dp = stat4_df.loc[stat4_df['Provider_Name']  == prov]
    
    dp_f = dp.loc[dp['has_buy_button'] == 'f']
    
    months = list(dp_f['Month'].unique())
    
    numb_products = [] 
    
    for month in months: 
        dp_s_d = dp_f.loc[dp_f['Month'] == month]
        
        n = len(dp_s_d['ean_13'].unique())
        numb_products.append(n)
        
        
    stats_p = pd.DataFrame({'Month': months,  
                            'Number of Products Missing Buy Button': numb_products})
    
    stats_p['Provider'] = prov
    
    Stats4_Data.append(stats_p)
    
    print(str(prov) + '  is done')

In [ ]:
#Concat Data and adding Domain 
Stat4_final = pd.concat(Stats4_Data)
Stat4_final['Domain'] = 'All Publishers'
Stat4_final.to_csv("2020_BuyButton.csv" , index = False)

In [ ]:
#Stats 3 : Columns needed {provider name , name , month , ean13 , found}

cl_stat5 = [] 
for chunk in chunk_list: 
    d = chunk[['Provider_Name', 'name' , 'Month' , 'ean_13', 'available_new' ]]
    cl_stat5.append(d)
    
    
stat5_df = pd.concat(cl_stat5)

In [ ]:
#Total unique products not available new - Amazon only
Stats5_Data = [] 
amazon = stat5_df.loc[stat5_df['Provider_Name']  == 'Amazon']
amazon_1 = amazon.loc[amazon['available_new'] == 'f']
months = list(amazon_1['Month'].unique())
numb_products = [] 
for month in months: 
    dp_s_d = amazon_1.loc[amazon_1['Month'] == month ]
    n = len(dp_s_d['ean_13'].unique())
    numb_products.append(n)
        
        
stats_a = pd.DataFrame({'Month': months, 
                        'Number of Products not Available New': numb_products})
    
stats_a['Provider'] = 'Amazon'
    
Stats5_Data.append(stats_a)

In [ ]:
#Concat Data and adding Domain 
Stat5_final = pd.concat(Stats5_Data)
Stat5_final['Domain'] = 'All Publishers'
Stat5_final.to_csv("2020_availablenew.csv" , index = False)

In [ ]:
#Stats 3 : Columns needed {provider name , name , month , ean13 , found}

cl_stat6 = [] 
for chunk in chunk_list: 
    d = chunk[['Provider_Name', 'name' , 'Month' , 'ean_13', 'list_price' ]]
    cl_stat6.append(d)
    
    
stat6_df = pd.concat(cl_stat6)

In [ ]:
#Divide data by provider 
providers = list(stat6_df['Provider_Name'].unique())

In [ ]:
Stats6_Data = [] 
for prov in providers: 
    dp = stat6_df.loc[stat6_df['Provider_Name']  == prov]
    
    dp1 = dp[['Month', 'Provider_Name' , 'ean_13' , 'list_price']]
    
    #
    
    eans = list(dp1['ean_13'].unique())
    
    data = []
    for i in eans: 
        d = dp1.loc[dp1['ean_13'] == i]
        d1 = d[['Month', 'list_price']]

        d2 = d1.set_index('Month').diff()
        d2 = d2.reset_index()

        d2['ean_13'] = i

        data.append(d2)
        
    final = pd.concat(data)
    
    months = list(final['Month'].unique())
    
    
    numbproducts = [] 
    
    
    for month in months: 
        d = final.loc[final['Month'] == month]
        dt = d.loc[d['list_price'] > 0]
        numbproducts.append(len(dt['ean_13'].unique()))
    
    
    dataF = pd.DataFrame({'Month':months, 
                         'Number of Products List Change': numbproducts})
    
    
    dataF['Provider'] = prov
    
    Stats6_Data.append(dataF)  
    
    print(str(prov) + 'is done')

In [ ]:
#Concat Data and adding Domain 
Stat6_final = pd.concat(Stats6_Data)
Stat6_final['Domain'] = 'All Publishers'
Stat6_final.to_csv("2020_listpricechange.csv" , index = False)

In [ ]:
#Saving Low Priority stats as an excel File 
with pd.ExcelWriter('FileName....') as writer:  #Set A name for the file 
    Stat3_final.to_excel(writer, sheet_name='Not Found')
    Stat4_final.to_excel(writer, sheet_name='Missing Buy Button')
    Stat5_final.to_excel(writer, sheet_name='Not Available New(Amazon)')
    Stat6_final.to_excel(writer, sheet_name='List Price Change')